# KHÁM PHÁ DỮ LIỆU

In [9]:
#Import các thư viện 
import numpy as np
import pandas as pd

### Đọc dữ liệu

Đọc từ đường dẫn ***./deccan gold mines ltd eod price.csv*** và lưu vào ***df***

In [10]:
path = './deccan gold mines ltd eod price.csv'
df = pd.read_csv(path)
df.head()

,Date,Open,High,Low,Close,WAP,No. of Shares,No. of Trades,Total Turnover,Deliverable Quantity,% Deli. Qty to Traded Qty,Spread H-L,Spread C-O
0,13-04-2023,50.47,50.47,47.86,49.15,49.35,122650,413,6053293,99475.0,81.10,2.61,-1.32
1,12-04-2023,49.98,51.00,49.30,50.07,49.99,193359,417,9665903,166630.0,86.18,1.70,0.09
2,11-04-2023,48.87,50.05,47.76,49.90,49.21,270964,509,13335488,208486.0,76.94,2.29,1.03
3,10-04-2023,51.49,51.49,48.00,48.16,49.90,339774,517,16954583,280083.0,82.43,3.49,-3.33
4,06-04-2023,48.40,49.04,47.65,49.04,48.80,245835,441,11996688,203526.0,82.79,1.39,0.64


### 4. Với mỗi cột, các giá trị (dạng số, dạng phân loại) được phân bố như thế nào?

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4812 entries, 0 to 4811
Data columns (total 13 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Date                       4812 non-null   object 
 1   Open                       4812 non-null   float64
 2   High                       4812 non-null   float64
 3   Low                        4812 non-null   float64
 4   Close                      4812 non-null   float64
 5   WAP                        4812 non-null   float64
 6   No. of Shares              4812 non-null   int64  
 7   No. of Trades              4812 non-null   int64  
 8   Total Turnover             4812 non-null   int64  
 9   Deliverable Quantity       4669 non-null   float64
 10  % Deli. Qty to Traded Qty  4669 non-null   float64
 11  Spread H-L                 4812 non-null   float64
 12  Spread C-O                 4812 non-null   float64
dtypes: float64(9), int64(3), object(1)
memory usage:

#### Nhận xét:
* Cột Ngày(Date) có dạng object là phù hợp cho các bước tính toán sau.
* Các cột còn lại (Open, Close,...) đều ở dạng Numerical. 
* Các cột đều có kiểu dữ liệu phù hợp nên không cần xử lý thêm.

#### Với mỗi cột dữ liệu: 
* Tỉ lệ % (từ 0 đến 100) các giá trị NaN (giá trị bị thiếu)
* Giá trị min
* Giá trị lower quartile Q1 (phân vị 25)
* Giá trị median Q2 (phân vị 50)
* Giá trị upper quartile Q3 (phân vị 75)
* Giá trị max

In [12]:
def get_val(col):
    NaN_Value = col.isna().sum()
    value = col.dropna()
    arr = []
    arr.append((NaN_Value/len(col))*100)
    arr.append(np.percentile(value, 0))
    arr.append(np.percentile(value, 25))
    arr.append(np.percentile(value, 50))
    arr.append(np.percentile(value, 75))
    arr.append(np.percentile(value, 100))
    return np.array(arr).round(2)

index_info=["missing_value", "min", "lower_quartile", "median", "upper_quartile", "max"]
info = {}
for i in df.columns:
    if df[i].dtype == np.float64:
        info[i] = get_val(df[i])   
columm_info_df =pd.DataFrame(info,index_info)
columm_info_df

,Open,High,Low,Close,WAP,Deliverable Quantity,% Deli. Qty to Traded Qty,Spread H-L,Spread C-O
missing_value,0.00,0.00,0.00,0.00,0.00,2.97,2.97,0.00,0.00
min,1.92,1.92,1.92,1.92,1.92,17.00,2.67,0.00,-13.60
lower_quartile,16.83,17.29,16.24,16.71,16.78,27524.00,71.63,0.75,-0.60
median,22.50,23.10,21.85,22.40,22.45,51465.00,81.42,1.15,-0.11
upper_quartile,33.90,34.61,32.82,33.70,33.79,96866.00,96.30,1.80,0.24
max,142.90,142.90,136.10,136.10,136.10,1460255.00,100.00,13.60,9.80


#### Nhận xét: 
* Đối với các cột dữ liệu dạng số, ta thấy min và max của mỗi cột cách nhau rất lớn, dữ liệu sẽ phân bố trong khoảng khá lớn.
* Các cột đa số đều không có giá trị bị thiếu, chỉ có 2 cột là **Deliverable Quantity** và **% Deli. Qty to Traded Qty** => Ta cần tiền xử lý để xử lý dữ liệu bị thiếu (missing data) 

### 5. Có cần phải tiền xử lý dữ liệu hay không và nếu có thì nhóm sinh viên cần phải xử lý như thế nào?

- Tiền xử lí dữ liệu (Data preprocessing) là quá trình chuẩn bị và biến đổi dữ liệu trước khi áp dụng các thuật toán máy học hoặc khai thác dữ liệu. 
Mục đích của tiền xử lí dữ liệu là làm cho dữ liệu trở nên chuẩn hóa và dễ dàng xử lý hơn, để có thể tạo ra các mô hình dự đoán chính xác hơn.Tiền xử lí dữ liệu là một bước quan trọng và không thể thiếu trong quá trình xây dựng các mô hình máy học hoặc khai thác dữ liệu, vì nó đảm bảo chất lượng và tính tin cậy của kết quả dự đoán.
- Các kỹ thuật tiền xử lý dữ liệu:
    - Làm sạch dữ liệu (data cleaning/cleansing)

        - Tóm tắt hoá dữ liệu: nhận diện đặc điểm chung của dữ liệu và sự hiện diện
        của nhiễu hoặc các phần tử kì dị (outliers)

        - Xử lý dữ liệu bị thiếu (missing data)

        - Xử lý dữ liệu bị nhiễu (noisy data)

    - Tích hợp dữ liệu (data integration)

        - Tích hợp lược đồ (schema integration) và so trùng đối tượng (object
        matching)
        - Vấn đề dư thừa (redundancy)

        - Phát hiện và xử lý mâu thuẫn giá trị dữ liệu (detection and resolution of data
        value conflicts)

    - Biến đổi dữ liệu (data transformation)

        - Làm trơn dữ liệu (smoothing)

        - Kết hợp dữ liệu (aggregation)

        - Tổng quát hóa dữ liệu (generalization)

        - Chuẩn hóa dữ liệu (normalization)

        - Xây dựng thuộc tính (attribute/feature construction)
    - Thu giảm dữ liệu (data reduction)

        - Kết hợp khối dữ liệu (data cube aggregation)

        - Chọn tập con các thuộc tính (attribute subset selection)

        - Thu giảm chiều (dimensionality reduction)

        - Thu giảm lượng (numerosity reduction)

        - Tạo phân cấp ý niệm (concept hierarchy generation) và rời rạc hóa (discretization)10

#### Xử lý dữ liệu bị thiếu
- Các cột đa số đều không có giá trị bị thiếu, chỉ có 2 cột là **Deliverable Quantity** và **% Deli. Qty to Traded Qty** => Ta cần tiền xử lý để xử lý dữ liệu bị thiếu (missing data). 
- Do khoảng cách giữa min và max của 2 cột này khá lớn, ta không nên thay giá trị bị thiếu bằng mean vì sẽ làm cho kết quả tính toán có thể không chính xác.
- Thay các giá trị bị thiếu bằng median để tránh ảnh hưởng nhiều đến tính toán sau này.

In [13]:
# Deliverable Quantity
med_Deli_Quantity = df["Deliverable Quantity"].median()
df["Deliverable Quantity"] = df["Deliverable Quantity"].fillna(med_Deli_Quantity)

# % Deli. Qty to Traded Qty
med_Deli_Qty = df["% Deli. Qty to Traded Qty"].median()
df["% Deli. Qty to Traded Qty"] = df["% Deli. Qty to Traded Qty"].fillna(med_Deli_Qty)

#### Các cột sau khi xử lý

In [14]:
def get_val(col):
    NaN_Value = col.isna().sum()
    value = col.dropna()
    arr = []
    arr.append((NaN_Value/len(col))*100)
    arr.append(np.percentile(value, 0))
    arr.append(np.percentile(value, 25))
    arr.append(np.percentile(value, 50))
    arr.append(np.percentile(value, 75))
    arr.append(np.percentile(value, 100))
    return np.array(arr).round(2)

index_info=["missing_value", "min", "lower_quartile", "median", "upper_quartile", "max"]
info = {}
for i in df.columns:
    if df[i].dtype == np.float64:
        info[i] = get_val(df[i])   
columm_info_df =pd.DataFrame(info,index_info)
columm_info_df

,Open,High,Low,Close,WAP,Deliverable Quantity,% Deli. Qty to Traded Qty,Spread H-L,Spread C-O
missing_value,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
min,1.92,1.92,1.92,1.92,1.92,17.00,2.67,0.00,-13.60
lower_quartile,16.83,17.29,16.24,16.71,16.78,28199.25,71.90,0.75,-0.60
median,22.50,23.10,21.85,22.40,22.45,51465.00,81.42,1.15,-0.11
upper_quartile,33.90,34.61,32.82,33.70,33.79,94606.00,94.90,1.80,0.24
max,142.90,142.90,136.10,136.10,136.10,1460255.00,100.00,13.60,9.80


#### Sau đó, ta hãy ghi df ra file ./deccan gold mines ltd eod price(da_xu_ly).csv

In [15]:
df.to_csv('./deccan gold mines ltd eod price(da_xu_ly).csv')